In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Load Markdown

Using `MarkdownNodeParser` for that

In [2]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path

md_docs = FlatReader().load_data(Path("./data/optisol_info.md"))

parser = MarkdownNodeParser()
nodes = parser.get_nodes_from_documents(md_docs)

In [3]:
len(nodes)

38

### Ingestion with Qdrant

In [4]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

def create_index(nodes, colection_name):
    vector_store = QdrantVectorStore(colection_name, client=client)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

In [ ]:
create_index(nodes, "optisol_first")

### Index to Retriever

In [7]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore("optisol_first", client=client)
index = VectorStoreIndex.from_vector_store(vector_store)

In [8]:
index

In [ ]:
retriever = index.as_retriever()
returned_nodes = retriever.retrieve("Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?")

In [ ]:
for node in returned_nodes:
    print(node.text)

## To ChatEngine

In [10]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# build index

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,    
    response_synthesizer=response_synthesizer,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)



In [11]:
query_engine.get_prompts()

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7fe507844b80>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

In [12]:
query_engine._response_synthesizer.get_prompts()

{'text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7fe507844b80>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, co

In [13]:
# query
response = query_engine.query("Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?")
print(response)

The current Chief Marketing Officer (CMO) of OptiSol Technologies is Emma Brown. Her background is in digital marketing and brand management.


In [ ]:
response

In [ ]:
response.source_nodes

### Helpers

In [14]:
def show_rag_nodes(source_nodes):
    for node in source_nodes:
        print(node.text)

def get_rag_response(query_response):
    return query_response.response

def get_rag_nodes(query_response):
    return query_response.source_nodes

## Change LLM

Global

In [ ]:
# from llama_index.llms.openai import OpenAI
# from llama_index.core import Settings

# Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

In [23]:
from llama_index.llms.openai import OpenAI


# define LLM
llm_mini = OpenAI(model="gpt-4o-mini", temperature=0.1)

# build index
# index_mini = KeywordTableIndex.from_documents(documents, llm=llm)

vector_store = QdrantVectorStore("optisol_first", client=client)
index_mini = VectorStoreIndex.from_vector_store(vector_store, llm=llm_mini)

# get response from query
query_engine_mini = index_mini.as_query_engine(response_mode="tree_summarize")
# response = query_engine.query(
#     "What did the author do after his time at Y Combinator?"
# )

In [24]:
questions = [
"Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?",
"Describe the specific initiatives OptiSol Technologies has implemented as part of their Green IT Initiative.",
"What are the key components of OptiSol Technologies' incident response approach in the event of a cybersecurity breach?",
"List and describe three major partnerships OptiSol Technologies has established for their cloud services.",
"What are the primary focus areas for OptiSol Technologies' future R&D efforts, and how do these areas align with current industry trends?"
]

In [25]:
for q in questions:
    response = query_engine_mini.query(q)
    print(get_rag_response(response))
    print("==="*15)

The current Chief Marketing Officer (CMO) of OptiSol Technologies is Emma Brown. Her background is in digital marketing and brand management, where she drives the company's marketing strategy and brand positioning.
OptiSol Technologies has implemented initiatives such as reducing the environmental impact of technology operations as part of their Green IT Initiative.
The key components of OptiSol Technologies' incident response approach in the event of a cybersecurity breach include identification, containment, eradication, recovery, and post-incident analysis.
OptiSol Technologies has established partnerships with Microsoft for cloud services and AI solutions, Amazon Web Services (AWS) for cloud infrastructure and services, and Google Cloud for AI and machine learning solutions.
The primary focus areas for OptiSol Technologies' future R&D efforts are Quantum Computing, Blockchain Technology, and Edge Computing. These areas align with current industry trends as they represent cutting-ed

### Change Prompt

In [29]:
from llama_index.core import PromptTemplate

# reset
# query_engine = index.as_query_engine(response_mode="tree_summarize")

# shakespeare!
new_summary_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "Remain descriptive."
    "But when listing anything, use good formatting.\n"
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)

query_engine_mini.update_prompts(
    {"response_synthesizer:summary_template": new_summary_tmpl}
)


This prompt update worked only when I initialized like this (otherwise it ignored the update prompt):

`query_engine_mini = index_mini.as_query_engine(response_mode="tree_summarize")`

In [30]:
for q in questions:
    response = query_engine_mini.query(q)
    print(get_rag_response(response))
    print("==="*15)

The current Chief Marketing Officer (CMO) of OptiSol Technologies is Emma Brown. Emma has a background in digital marketing and brand management, and she oversees all marketing and communications efforts at OptiSol. She drives the company's marketing strategy and brand positioning.
As part of their Green IT Initiative, OptiSol Technologies has implemented the following specific initiatives:

- Reducing the environmental impact of technology operations.
- Implementing sustainable practices in all operations.
The key components of OptiSol Technologies' incident response approach in the event of a cybersecurity breach include:

1. Identification
2. Containment
3. Eradication
4. Recovery
5. Post-incident analysis

Additionally, OptiSol Technologies also provides incident response planning and training to help businesses prepare for and respond to future incidents effectively.
1. **Technology Partnerships:**  
   - **Microsoft:** OptiSol Technologies has a strategic partnership with Microso

In [27]:
query_engine_mini.get_prompts()

{'response_synthesizer:summary_template': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, When listing anything, use bullet points.\nQuery: {query_str}\nAnswer: ')}

In [ ]:
query_engine_mini

In [ ]:
chat_engine = index.as_chat_engine()

In [ ]:
response = chat_engine.query("Who is the current Chief Marketing Officer (CMO) of OptiSol Technologies, and what is their background?")

In [ ]:
len(response.source_nodes), response.response

In [ ]:
source_nodes = response.source_nodes

In [ ]:
for node in source_nodes:
    print(node.text)

In [ ]:
q2response = chat_engine.query("Describe the specific initiatives OptiSol Technologies has implemented as part of their Green IT Initiative.")

In [ ]:
get_rag_response(q2response)

In [ ]:
show_rag_nodes(get_rag_nodes(q2response))

### Prompt Rewrite

If I decide to change the prompt, I can go here:

- [Source](https://docs.llamaindex.ai/en/stable/examples/prompts/prompt_mixin/#customize-the-prompt)
